<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Lesson6_MetricsAndFeaturesAdv/RegularizationsPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Неклассические регуляризации

### **Elastic Net**

**Elastic Net** — это метод регуляризации, который комбинирует $\ell_1$ (LASSO) и $\ell_2$ (Ridge) регуляризации через линейное взвешивание.  

Формула Elastic Net:
$$
R_{\text{Elastic Net}}(\mathbf{w}) = \alpha \|\mathbf{w}\|_1 + (1 - \alpha) \frac{1}{2} \|\mathbf{w}\|_2^2,
$$
где:
- $\|\mathbf{w}\|_1 = \sum |w_i|$ — $\ell_1$-норма, отвечающая за разреженность (спарсити);
- $\|\mathbf{w}\|_2^2 = \sum w_i^2$ — $\ell_2$-норма, обеспечивающая устойчивость к шуму;
- $\alpha \in [0, 1]$ — гиперпараметр, задающий баланс между $\ell_1$- и $\ell_2$-нормами.

#### **Основные свойства Elastic Net:**
1. **Смешивание разреженности и устойчивости:**
   - $\ell_1$-норма отбирает признаки, зануляя неважные веса.
   - $\ell_2$-норма уменьшает веса всех признаков, предотвращая нестабильность и переобучение.
   
2. **Скалируемость:**
   - Elastic Net хорошо работает с высокоразмерными данными, особенно в случаях, когда признаки сильно коррелированы.

3. **Простая оптимизация:**
   - Функция Elastic Net является выпуклой, что упрощает её минимизацию.

---

### **Huber-подобная регуляризация**

**Huber-подобная регуляризация** также объединяет $\ell_1$- и $\ell_2$ -свойства, но делает это адаптивно, в зависимости от величины весов:

Формула:
$$
R_{\delta}(\mathbf{w}) = \sum_{i=1}^n
\begin{cases}
\frac{1}{2} w_i^2 & \text{если } |w_i| \leq \delta, \\
\delta (|w_i| - \frac{1}{2} \delta) & \text{если } |w_i| > \delta.
\end{cases}
$$

#### **Основные свойства Huber-регуляризации:**
1. **Адаптивность:**
   - Для малых весов $(|w_i| \leq \delta)$ используется $\ell_2$-штраф, что делает веса устойчивыми к шуму.
   - Для больших весов $(|w_i| > \delta)$ используется $\ell_1$-штраф, чтобы занулить незначимые признаки.

2. **Устойчивость к выбросам:**
   - Линейное штрафование больших весов делает $\ell_1$ регуляризацию менее чувствительной к выбросам.

3. **Задачи с выбросами:**
   - Huber-подобная регуляризация чаще применяется в задачах с аномалиями, где нужно адаптивное поведение.

---

### **Ключевые различия**

| **Характеристика**               | **Elastic Net**                                            | **Huber-подобная регуляризация**                          |
|-----------------------------------|-----------------------------------------------------------|----------------------------------------------------------|
| **Баланс $\ell_1$ и $\ell_2$** | Линейная комбинация двух норм                 | Адаптивное переключение между $\ell_1$ и $\ell_2$    |
| **Цель**                          | Обеспечить разреженность и устойчивость                   | Устойчивость к выбросам и шуму                           |
| **Способ применения**             | Все признаки сразу штрафуются линейной комбинацией норм   | Штраф зависит от значения весов $|w_i|$              |
| **Параметры**                     | Один параметр \(\alpha\) для баланса $\ell_1$ и $\ell_2$ | Один параметр $\delta$ для порога переключения         |
| **Выпуклость функции**            | Выпуклая (легко оптимизируется)                          | Не всегда выпуклая (зависит от аппроксимации)            |
| **Работа с выбросами**            | Чувствителен к выбросам                                   | Устойчив к выбросам (штрафует их линейно)                |
| **Применение**                    | Высокая корреляция признаков, отбор признаков             | Задачи с шумными или выбросоопасными данными             |

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from itertools import combinations
from math import log

# 1. Генерация данных для многомерной линейной регрессии
np.random.seed(42)
n_samples, n_features, n_targets = 100, 5, 1

X = np.random.rand(n_samples, n_features)
true_coefs = np.array([
    [3, 0, 0, 2, 0],  # Для первой компоненты Y
    [0, -1, 0, 0, 1],  # Для второй компоненты Y
    [2, 0, 1, 0, 0]    # Для третьей компоненты Y
])

Y = X @ true_coefs.T + np.random.normal(scale=1.0, size=(n_samples, n_targets))

# 2. Добавление выбросов (5% от общего числа наблюдений)
n_outliers = int(0.05 * n_samples)
outlier_indices = np.random.choice(np.arange(n_samples), n_outliers, replace=False)

for i in outlier_indices:
    Y[i] += np.random.normal(scale=200, size=n_targets)  # Добавляем большие случайные значения

# Преобразование данных в DataFrame для удобства
df = pd.DataFrame(X, columns=[f"Feature_{i}" for i in range(n_features)])
for i in range(n_targets):
    df[f"Target_{i}"] = Y[:, i]

In [3]:
df.head()

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Target_0
0,0.374540,0.950714,0.731994,0.598658,0.156019,2.662693
1,0.155995,0.058084,0.866176,0.601115,0.708073,3.546384
2,0.020584,0.969910,0.832443,0.212339,0.181825,1.436856
3,0.183405,0.304242,0.524756,0.431945,0.291229,0.837200
4,0.611853,0.139494,0.292145,0.366362,0.456070,1.669868


In [4]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split

# 2. Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Target_0'], axis=1), df['Target_0'], test_size=0.2, random_state=42)

# 3. Elastic Net
elastic_net = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=42)  # l1_ratio: баланс L1 и L2
elastic_net.fit(X_train, y_train)

# Предсказание и оценка
y_pred_en = elastic_net.predict(X_test)
mse_en = mean_squared_error(y_test, y_pred_en)

print(f"Mean Squared Error (Elastic Net): {mse_en:.3f}")

Mean Squared Error (Elastic Net): 8151.304


In [5]:
# 2. Определение функции для Huber-like регуляризации
def huber_regularization(beta, delta=1.0):
    return np.sum(delta * (np.abs(beta) - 0.5 * delta))

# 3. Обучение модели с использованием MSE + Huber-like регуляризации
class RidgeWithHuberReg(Ridge):
    def __init__(self, alpha=1.0, delta=1.0):
        super().__init__(alpha=alpha)
        self.delta = delta

    def fit(self, X, y):
        super().fit(X, y)
        # Применяем Huber-like регуляризацию к коэффициентам
        huber_penalty = huber_regularization(self.coef_, delta=self.delta)
        self.coef_ -= self.alpha * huber_penalty  # Регуляризация по весам

        return self

# Обучаем модель с Huber-like регуляризацией
model = RidgeWithHuberReg(alpha=0.01, delta=0.5)
model.fit(X_train, y_train)

# 4. Оценка модели на тестовых данных
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Test MSE with Huber-like regularization: {mse:.4f}")

Test MSE with Huber-like regularization: 8282.0194


Есть еще huber-регрессия, но это не регуляризация!

In [6]:
from sklearn.linear_model import HuberRegressor

# 4. Huber-регрессия
huber = HuberRegressor(alpha=0.5, max_iter=1000)
huber.fit(X_train, y_train)

# Предсказание и оценка
y_pred_huber = huber.predict(X_test)
mse_huber = mean_squared_error(y_test, y_pred_huber)

print(f"Mean Squared Error (Huber): {mse_huber:.3f}")

Mean Squared Error (Huber): 8080.887


# Подбор гиперпараметров по кросс-валидации

In [7]:
from sklearn.model_selection import GridSearchCV

params = {'alpha' : [0.01, 0.1, 0.5, 1, 10]}

model = HuberRegressor(max_iter=1000)

gs = GridSearchCV(model, params, cv=3, scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=HuberRegressor(max_iter=1000),
             param_grid={'alpha': [0.01, 0.1, 0.5, 1, 10]},
             scoring='neg_mean_squared_error')

In [10]:
-gs.best_score_, gs.best_estimator_, gs.best_params_

(3166.866866716891, HuberRegressor(alpha=10, max_iter=1000), {'alpha': 10})

In [11]:
y_pred_huber_tuned = gs.predict(X_test)
mse_huber_tuned = mean_squared_error(y_test, y_pred_huber_tuned)

print(f"Mean Squared Error (Huber, tuned): {mse_huber_tuned:.3f}")

Mean Squared Error (Huber, tuned): 8049.108


# Информационные критерии

**AIC** (информационный критерий Акаике) и **BIC** (байесовский информационный критерий) — это два статистических критерия, которые используются для выбора модели среди множества кандидатов. Оба критерия оценивают баланс между точностью модели (ее способностью подгонять данные) и сложностью модели (числом параметров).

Они являются **критериями информации**, что означает, что их задача — минимизировать потерю информации при выборе модели. В идеале мы хотим выбрать такую модель, которая точно описывает данные, но не является излишне сложной.

---

### 1. **AIC (Информационный критерий Акаике)**

**Формула:**

$$
\text{AIC} = -2 \ln(L) + 2k
$$

Где:
- $ L $ — функция правдоподобия модели (то есть вероятность наблюдаемых данных при условии выбранной модели).
- $ k $ — количество параметров в модели (например, количество коэффициентов в линейной регрессии).
- $ \ln $ — натуральный логарифм.

**Интерпретация AIC:**
- **Меньшее значение AIC** означает лучшую модель, так как оно отражает баланс между хорошей подгонкой данных (чем выше правдоподобие $ L $, тем меньше AIC) и простотой модели (чем больше параметров, тем больше значение AIC).
- AIC штрафует модели с большим количеством параметров, тем самым предотвращая переобучение.

**Использование AIC:**
- AIC используется для выбора между несколькими моделями, которые описывают одни и те же данные. Модель с наименьшим значением AIC считается наиболее оптимальной, так как она балансирует точность и сложность.

---

### 2. **BIC (Байесовский информационный критерий)**

**Формула:**

$$
\text{BIC} = -2 \ln(L) + k \ln(n)
$$

Где:
- $ L $ — функция правдоподобия модели (также, как и в AIC).
- $ k $ — количество параметров модели.
- $ n $ — количество наблюдений (размер выборки).

**Интерпретация BIC:**
- **BIC** схож с AIC, но включает в себе дополнительный фактор — размер выборки $ n $. Этот фактор увеличивает штраф за сложность модели с ростом размера выборки. Таким образом, BIC более строго штрафует за добавление лишних параметров, чем AIC, особенно при большом объеме данных.
- Меньшее значение BIC указывает на более подходящую модель.

**Использование BIC:**
- BIC также используется для выбора лучшей модели среди нескольких. Однако, в отличие от AIC, BIC склонен отдавать предпочтение более простым моделям, особенно если количество наблюдений велико, поскольку $ \ln(n) $ растет с увеличением размера выборки.

In [ ]:
# Преобразование данных в DataFrame для удобства
df_train = pd.DataFrame(X_train, columns=[f"Feature_{i}" for i in range(n_features)])
for i in range(n_targets):
    df_train[f"Target_{i}"] = y_train

# 2. Функция для расчета AIC и BIC для многомерной регрессии
def calculate_aic_bic_multi(model, X, Y):
    n, k = X.shape[0], X.shape[1]  # Число наблюдений и признаков
    n_targets = Y.shape[1]  # Число целевых переменных
    y_pred = model.predict(X)
    residual_sum_of_squares = np.sum((Y - y_pred) ** 2)

    # Средняя ошибка для всех компонент целевой переменной
    mse = mean_squared_error(Y, y_pred, multioutput='uniform_average')
    log_likelihood = -n * n_targets / 2 * (np.log(2 * np.pi * mse) + 1)

    # Количество параметров = (число признаков + 1) * число целевых переменных
    total_params = n_targets * (k + 1)

    aic = 2 * total_params - 2 * log_likelihood
    bic = total_params * log(n) - 2 * log_likelihood

    return aic, bic

# 3. Перебор комбинаций признаков для минимизации AIC/BIC
min_aic, min_bic = float('inf'), float('inf')
best_aic_model, best_bic_model = None, None

for num_features in range(1, n_features + 1):
    for feature_subset in combinations(df_train.columns[:n_features], num_features):
        # Подготовка данных
        X_subset = df_train[list(feature_subset)].values
        Y_subset = df_train[[f"Target_{i}" for i in range(n_targets)]].values

        # Обучение модели
        model = LinearRegression()
        model.fit(X_subset, Y_subset)

        # Расчет AIC и BIC
        aic, bic = calculate_aic_bic_multi(model, X_subset, Y_subset)

        # Обновление лучших моделей
        if aic < min_aic:
            min_aic = aic
            best_aic_model = (model, feature_subset)

        if bic < min_bic:
            min_bic = bic
            best_bic_model = (model, feature_subset)

# 4. Вывод результатов
print(f"Best AIC: {min_aic:.2f} using features {best_aic_model[1]}")
print(f"Best BIC: {min_bic:.2f} using features {best_bic_model[1]}")

# 5. Оценка на тестовой выборке
X_test_np = X_test.values

# Access columns by index using NumPy-style slicing
X_test_aic = X_test_np[:, [int(feature.split('_')[1]) for feature in best_aic_model[1]]]
X_test_bic = X_test_np[:, [int(feature.split('_')[1]) for feature in best_bic_model[1]]]

aic_test_mse = mean_squared_error(y_test, best_aic_model[0].predict(X_test_aic), multioutput='uniform_average')
bic_test_mse = mean_squared_error(y_test, best_bic_model[0].predict(X_test_bic), multioutput='uniform_average')

print(f"Test MSE for AIC-selected model: {aic_test_mse:.4f}")
print(f"Test MSE for BIC-selected model: {bic_test_mse:.4f}")

Best AIC: 874.91 using features ('Feature_2',)
Best BIC: 879.68 using features ('Feature_2',)
Test MSE for AIC-selected model: 7749.0920
Test MSE for BIC-selected model: 7749.0920
